## Criando uma RAG

In [1]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI
model = ChatOpenAI(model="gpt-4o-mini")

In [9]:
caminhos = [
    "../pdfes/Explorando a API da OpenAI.pdf",
    "../pdfes/Explorando o Universo das IAs com Hugging Face.pdf",
    "../pdfes/Miyamoto Musashi - Gorin no Sho - O Livro dos Cinco Elementos.pdf",
    "../pdfes/ultra-aprendizado-ultralearning-scott-young.pdf"
]

In [ ]:
paginas = []
for pdf in caminhos:
    loader = PyPDFLoader(pdf)
    paginas.extend(loader.load_and_split())

print(len(paginas))

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, 
    chunk_overlap=200,
    separators=[".", "!","\n\n"]
)

chunks = text_splitter.split_documents(paginas)

def limpa_texto(texto):
    return texto.replace("\n", " ").strip()

for chunk in chunks:
    chunk.page_content = limpa_texto(chunk.page_content)

print(chunks[500].page_content)

In [30]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(
    chunks,
    embedding=embeddings
)

In [ ]:
retriever = vectorstore.as_retriever(
    search_type="mmr", 
    search_kwargs={"k": 5}
)

retriever.invoke("Como ser um samurai?")

## Criando uma Chain



In [34]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

template = ChatPromptTemplate.from_template(
    """Você é um assistente de IA que responde perguntas de acordo com o contexto fornecido.

    Contexto: {contexto}
    Pergunta: {pergunta}
    """
)
output_parser = StrOutputParser()

In [ ]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

setup_and_retriever = RunnableParallel({
    "pergunta": RunnablePassthrough(),
    "contexto": retriever
})

chain = setup_and_retriever | template | model | output_parser

chain.invoke("Como ser um samurai?")

#### Entendendo a Chain


In [ ]:
RunnablePassthrough().invoke("Como ser um samurai?") # Passa a pergunta para o modelo

retriever.invoke("Como ser um samurai?") # Passa a pergunta para o retriever

RunnableParallel({  #É uma função que recebe um dicionário de inputs e retorna um dicionário de outputs resumidamente
    "pergunta": RunnablePassthrough(),
    "contexto": retriever
}).invoke("Como ser um samurai?") # Passa a pergunta para o retriever e retorna o contexto

#### Criando uma alternativa não paralelizavel




In [ ]:
setup_dict = {
    "pergunta": RunnablePassthrough(), 
    "contexto": retriever
}
chain = setup_dict | template | model | output_parser
chain.invoke("Como ser um samurai?")

#### Utilizando Fallback

O fallback é um mecanismo de segurança que permite ter um "plano B" quando algo falha na execução da chain. Em outras palavras, é uma alternativa que entra em ação quando o componente principal não funciona como esperado.

No contexto do LangChain, você pode usar fallbacks para:
Modelos de linguagem (LLMs):
Se um modelo falhar ou ficar indisponível, outro modelo pode ser usado como backup
Por exemplo: se o GPT-4 falhar, pode usar o GPT-3.5 como fallback

Embeddings:
Se um serviço de embeddings falhar, outro pode ser usado
Por exemplo: se OpenAI Embeddings falhar, pode usar HuggingFace Embeddings

Retrievers:
Se um método de recuperação falhar, outro pode ser usado
Por exemplo: se a busca por similaridade falhar, pode usar busca por palavras-chave


In [ ]:
from langchain_openai import OpenAI
from langchain.prompts import PromptTemplate

llm = OpenAI(model="gpt-3.5-turbo-instruct")
prompt = PromptTemplate.from_template("Resuma o seguinte texto: {texto}")

chain_pequena = prompt | llm
chain_pequena.invoke({"texto":"Oi, tudo bem?"})

In [21]:
from langchain_openai import ChatOpenAI

llm1 = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.0)
llm2 = ChatOpenAI(model="gpt-4o", temperature=0.0)
llm3 = ChatOpenAI(model="gpt-4o-mini")
llm4 = ChatOpenAI(model="o3-mini-2025-01-31")
llm5 = ChatOpenAI(model="o1-2024-12-17")

prompt = PromptTemplate.from_template("Resuma o seguinte texto: {texto}")

chain_pequena = prompt | llm4
chain_pequena.invoke({"texto":"Oi, tudo bem?"})

AIMessage(content='O texto é uma saudação informal que inicia uma conversa.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 279, 'prompt_tokens': 18, 'total_tokens': 297, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 256, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'o3-mini-2025-01-31', 'system_fingerprint': 'fp_42bfad963b', 'finish_reason': 'stop', 'logprobs': None}, id='run-2c18a373-8845-421f-87e7-f2134fe21a22-0', usage_metadata={'input_tokens': 18, 'output_tokens': 279, 'total_tokens': 297, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 256}})

In [ ]:
chain_pequena.invoke({"texto":"Oi, tudo bem?" *1000})

In [23]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

llm = ChatOpenAI(model="gpt-3.5-turbo-0125")
prompt = ChatPromptTemplate.from_template("Resuma o seguinte texto: {texto}")

chain_grande = prompt | llm | StrOutputParser()
chain_grande.invoke({"texto":"Oi, tudo bem?" *1000})

'Oi, tudo bem?\n\nO texto traz uma repetição constante da mesma pergunta, "Oi, tudo bem?", sem nenhuma variação ou acréscimo de informações adicionais.'

In [24]:
chain_fallback = chain_pequena.with_fallbacks([chain_grande])
chain_fallback.invoke({"texto":"Oi, tudo bem?" *1000})

AIMessage(content='O texto consiste na repetição contínua da saudação "Oi, tudo bem?", sem desenvolvimento de outras ideias ou informações adicionais.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 165, 'prompt_tokens': 5013, 'total_tokens': 5178, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 128, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'o3-mini-2025-01-31', 'system_fingerprint': 'fp_42bfad963b', 'finish_reason': 'stop', 'logprobs': None}, id='run-323edc7c-7ecd-470f-b9c1-9df756b31c23-0', usage_metadata={'input_tokens': 5013, 'output_tokens': 165, 'total_tokens': 5178, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 128}})

#### Fallback de Formatação


In [ ]:
from langchain_openai import OpenAI
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser 
from langchain.output_parsers import DatetimeOutputParser

prompt = PromptTemplate.from_template(
    "Qual foi a data do evento {evento} (no formato %Y-%m-%dT%H:%M:%S.%Z - Retorne apenas o valor solicitado)?"
)

chain_instruct = prompt | OpenAI(model="gpt-3.5-turbo-instruct") | DatetimeOutputParser()
chain_instruct.invoke({"evento":"Qual foi a data do final da copa de 2002?"})

In [ ]:
chain_turbo = prompt | ChatOpenAI(model="gpt-3.5-turbo-0125") | DatetimeOutputParser()
chain_turbo.invoke({"evento":"Qual foi a data do final da copa de 2002?"})

In [ ]:
chain_fallback = chain_instruct.with_fallbacks([chain_turbo])
chain_fallback.invoke({"evento":"Qual foi a data do final da copa de 2002?"})

In [25]:
from langchain.chat_models import ChatOpenAI

chat_4o = ChatOpenAI(model="gpt-4o")

try:
    response = chat_4o.invoke("Teste de conexão")
    print(response)
except Exception as e:
    print(f"Erro direto na OpenAI: {e}")


C:\Users\dehbo\AppData\Local\Temp\ipykernel_14272\221264837.py:3: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  chat_4o = ChatOpenAI(model="gpt-4o")


content='Parece que sua conexão está funcionando corretamente. Como posso ajudá-lo hoje?' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 10, 'total_tokens': 26, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_90d33c15d4', 'finish_reason': 'stop', 'logprobs': None} id='run-b57d7623-7e7e-4020-9d3f-ac0c28256350-0'
